# Ingest

Ingest all *.rdf files recursively, convert to a structured numpy array and upload to S3 for easy public download via numpy.DataSource().open(url)

See https://github.com/braingeneers/braingeneers/wiki/File-Formats for details on the source file format

Note: The original Intan code saves the data as float64 which effectively increaes the size of the data by 4x vs its original source as 16 bit samples. The below code is modified to keep things in 16 bit form and as a result does not scale the amplifier_data structure leaving that to the analysis phase. There is still significant overhead added vs. the raw file (82M vs. 44M) but its much better then just storing the float64

In [10]:
import os
import glob
import json
import datetime
import numpy as np

In [30]:
#! /bin/env python
#
# Michael Gibson 17 July 2015
# Modified Adrian Foy Sep 2018
# Modified to leave amplifier_data as 16 bit unsigned int and to be less chatty
# This is all a bit of a hack IMHO and needs to be re-structured/cleaned up from the source intan code

import sys, struct, math, os, time
import numpy as np

from intanutil.read_header import read_header
from intanutil.get_bytes_per_data_block import get_bytes_per_data_block
from intanutil.read_one_data_block import read_one_data_block
from intanutil.notch_filter import notch_filter
from intanutil.data_to_result import data_to_result

def read_data(filename):
    """
    Reads Intan Technologies RHD2000 data file generated by evaluation board GUI.
    Data are returned in a dictionary, for future extensibility.
    """

    tic = time.time()
    fid = open(filename, 'rb')
    filesize = os.path.getsize(filename)

    header = read_header(fid)

    print('{} amplifier channels'.format(header['num_amplifier_channels']))
    print('{} auxiliary input channels'.format(header['num_aux_input_channels']))
    print('{} supply voltage channels'.format(header['num_supply_voltage_channels']))
    print('{} board ADC channels'.format(header['num_board_adc_channels']))
    print('{} board digital input channels'.format(header['num_board_dig_in_channels']))
    print('{} board digital output channels'.format(header['num_board_dig_out_channels']))
    print('{} temperature sensors channels'.format(header['num_temp_sensor_channels']))

    # Determine how many samples the data file contains.
    bytes_per_block = get_bytes_per_data_block(header)

    # How many data blocks remain in this file?
    data_present = False
    bytes_remaining = filesize - fid.tell()
    if bytes_remaining > 0:
        data_present = True

    if bytes_remaining % bytes_per_block != 0:
        raise Exception('Something is wrong with file size : should have a whole number of data blocks')

    num_data_blocks = int(bytes_remaining / bytes_per_block)

    num_amplifier_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_aux_input_samples = int((header['num_samples_per_data_block'] / 4) * num_data_blocks)
    num_supply_voltage_samples = 1 * num_data_blocks
    num_board_adc_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_board_dig_in_samples = header['num_samples_per_data_block'] * num_data_blocks
    num_board_dig_out_samples = header['num_samples_per_data_block'] * num_data_blocks

    record_time = num_amplifier_samples / header['sample_rate']

    if data_present:
        print('File contains {:0.3f} seconds of data.  Amplifiers were sampled at {:0.2f} kS/s.'.format(record_time, header['sample_rate'] / 1000))
    else:
        print('Header file contains no data.  Amplifiers were sampled at {:0.2f} kS/s.'.format(header['sample_rate'] / 1000))

    if data_present:
        # Pre-allocate memory for data.
        print('')
        print('Allocating memory for data...')

        data = {}
        if (header['version']['major'] == 1 and header['version']['minor'] >= 2) or (header['version']['major'] > 1):
            data['t_amplifier'] = np.zeros(num_amplifier_samples, dtype=np.int)
        else:
            data['t_amplifier'] = np.zeros(num_amplifier_samples, dtype=np.uint)

        # NOTE: Changed from uint to uint16
        data['amplifier_data'] = np.zeros([header['num_amplifier_channels'], num_amplifier_samples], dtype=np.uint16)
        data['aux_input_data'] = np.zeros([header['num_aux_input_channels'], num_aux_input_samples], dtype=np.uint)
        data['supply_voltage_data'] = np.zeros([header['num_supply_voltage_channels'], num_supply_voltage_samples], dtype=np.uint)
        data['temp_sensor_data'] = np.zeros([header['num_temp_sensor_channels'], num_supply_voltage_samples], dtype=np.uint)
        data['board_adc_data'] = np.zeros([header['num_board_adc_channels'], num_board_adc_samples], dtype=np.uint)
        
        # by default, this script interprets digital events (digital inputs and outputs) as booleans
        # if unsigned int values are preferred(0 for False, 1 for True), replace the 'dtype=np.bool' argument with 'dtype=np.uint' as shown
        # the commented line below illustrates this for digital input data; the same can be done for digital out
        
        #data['board_dig_in_data'] = np.zeros([header['num_board_dig_in_channels'], num_board_dig_in_samples], dtype=np.uint)
        data['board_dig_in_data'] = np.zeros([header['num_board_dig_in_channels'], num_board_dig_in_samples], dtype=np.bool)
        data['board_dig_in_raw'] = np.zeros(num_board_dig_in_samples, dtype=np.uint)
        
        data['board_dig_out_data'] = np.zeros([header['num_board_dig_out_channels'], num_board_dig_out_samples], dtype=np.bool)
        data['board_dig_out_raw'] = np.zeros(num_board_dig_out_samples, dtype=np.uint)

        # Read sampled data from file.
        print('Reading data from file...')

        # Initialize indices used in looping
        indices = {}
        indices['amplifier'] = 0
        indices['aux_input'] = 0
        indices['supply_voltage'] = 0
        indices['board_adc'] = 0
        indices['board_dig_in'] = 0
        indices['board_dig_out'] = 0

        print_increment = 10
        percent_done = print_increment
        for i in range(num_data_blocks):
            read_one_data_block(data, header, indices, fid)

            # Increment indices
            indices['amplifier'] += header['num_samples_per_data_block']
            indices['aux_input'] += int(header['num_samples_per_data_block'] / 4)
            indices['supply_voltage'] += 1
            indices['board_adc'] += header['num_samples_per_data_block']
            indices['board_dig_in'] += header['num_samples_per_data_block']
            indices['board_dig_out'] += header['num_samples_per_data_block']            

            fraction_done = 100 * (1.0 * i / num_data_blocks)
            if fraction_done >= percent_done:
                print('.', end='', flush=True)
#                 print('{}% done...'.format(percent_done))
                percent_done = percent_done + print_increment

        # Make sure we have read exactly the right amount of data.
        bytes_remaining = filesize - fid.tell()
        if bytes_remaining != 0: raise Exception('Error: End of file not reached.')

    # Close data file.
    fid.close()

    if (data_present):
        print('Parsing data...')

        # Extract digital input channels to separate variables.
        for i in range(header['num_board_dig_in_channels']):
            data['board_dig_in_data'][i, :] = np.not_equal(np.bitwise_and(data['board_dig_in_raw'], (1 << header['board_dig_in_channels'][i]['native_order'])), 0)

        # Extract digital output channels to separate variables.
        for i in range(header['num_board_dig_out_channels']):
            data['board_dig_out_data'][i, :] = np.not_equal(np.bitwise_and(data['board_dig_out_raw'], (1 << header['board_dig_out_channels'][i]['native_order'])), 0)

        # Scale voltage levels appropriately.
        # NOTE: Commented out to reduce size of file by 4x by storing 16 bit ints in the resulting .npy
#         data['amplifier_data'] = np.multiply(0.195, (data['amplifier_data'].astype(np.int32) - 32768))      # units = microvolts
        data['aux_input_data'] = np.multiply(37.4e-6, data['aux_input_data'])               # units = volts
        data['supply_voltage_data'] = np.multiply(74.8e-6, data['supply_voltage_data'])     # units = volts
        if header['eval_board_mode'] == 1:
            data['board_adc_data'] = np.multiply(152.59e-6, (data['board_adc_data'].astype(np.int32) - 32768)) # units = volts
        elif header['eval_board_mode'] == 13:
            data['board_adc_data'] = np.multiply(312.5e-6, (data['board_adc_data'].astype(np.int32) - 32768)) # units = volts
        else:
            data['board_adc_data'] = np.multiply(50.354e-6, data['board_adc_data'])           # units = volts
        data['temp_sensor_data'] = np.multiply(0.01, data['temp_sensor_data'])               # units = deg C

        # Check for gaps in timestamps.
        num_gaps = np.sum(np.not_equal(data['t_amplifier'][1:]-data['t_amplifier'][:-1], 1))
        assert num_gaps == 0  # We don't handle missing samples in all our downstream analysis

        # Scale time steps (units = seconds).
        data['t_amplifier'] = data['t_amplifier'] / header['sample_rate']
        data['t_aux_input'] = data['t_amplifier'][range(0, len(data['t_amplifier']), 4)]
        data['t_supply_voltage'] = data['t_amplifier'][range(0, len(data['t_amplifier']), header['num_samples_per_data_block'])]
        data['t_board_adc'] = data['t_amplifier']
        data['t_dig'] = data['t_amplifier']
        data['t_temp_sensor'] = data['t_supply_voltage']

        # If the software notch filter was selected during the recording, apply the
        # same notch filter to amplifier data here.
        assert header['notch_filter_frequency'] == 0
        if header['notch_filter_frequency'] > 0:
            print('Applying notch filter...')

            print_increment = 10
            percent_done = print_increment
            for i in range(header['num_amplifier_channels']):
                data['amplifier_data'][i,:] = notch_filter(data['amplifier_data'][i,:], header['sample_rate'], header['notch_filter_frequency'], 10)

                fraction_done = 100 * (i / header['num_amplifier_channels'])
                if fraction_done >= percent_done:
                    print('{}% done...'.format(percent_done))
                    percent_done += print_increment
    else:
        data = [];

    # Move variables to result struct.
    result = data_to_result(header, data, data_present)

    print('Done!  Elapsed time: {0:0.1f} seconds'.format(time.time() - tic))
    return result

In [36]:
os.chdir("/public/groups/braingeneers/archive")
os.makedirs("derived/2018-12-11", exist_ok=True)

for path in glob.glob("original/2018-12-11/**/*.rhd"):
    print(path)
    data = read_data(path)
    
    signal = data["amplifier_data"]
    metadata = data
    
    metadata["date"] = "2018-12-11"  # ISO 8601

    del metadata["t_supply_voltage"]
    del metadata["t_amplifier"]
    # spike_triggers 0
    # notes 0
    # supply_voltage_channels 0
    # amplifier_channels 0
    del metadata["amplifier_data"]
    del metadata["t_aux_input"]
    del metadata["aux_input_channels"]
    del metadata["aux_input_data"]
    del metadata["t_temp_sensor"]
    # frequency_parameters 0
    del metadata["supply_voltage_data"]

    metadata["source_file_path"] = path  # Add annotation of source file path and name relative to archive/

    dest = "derived/2018-12-11/{2}-{3}".format(*path.replace(" ","-").split("/"))[:-4]
    print("Saving numpy and json to {}".format(dest))
    np.save(dest + ".npy", signal)
    with open(dest + ".json", "w") as f:
        json.dump(metadata, f)
    print()

original/2018-12-11/organoid 1/recording background 100Hz filter_181208_130030.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
16 amplifier channels
3 auxiliary input channels
1 supply voltage channels
0 board ADC channels
0 board digital input channels
0 board digital output channels
1 temperature sensors channels
File contains 41.112 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
.........Parsing data...
Done!  Elapsed time: 18.0 seconds
Saving numpy and json to derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030

original/2018-12-11/organoid 1/recording_1_181208_122247.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
17 amplifier channels
3 auxiliary input channels
1 supply voltage channels
0 board ADC channels
0 board digital input channels
0 board digital output channels
1 temperature sensors channels
File contains 60.012 s

.........Parsing data...
Done!  Elapsed time: 26.5 seconds
Saving numpy and json to derived/2018-12-11/organoid-2-recording_3-wt_100hz-filter_181208_135611

original/2018-12-11/organoid 2/recording_3 wt_100hz filter_181208_135711.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
16 amplifier channels
3 auxiliary input channels
1 supply voltage channels
0 board ADC channels
0 board digital input channels
0 board digital output channels
1 temperature sensors channels
File contains 42.912 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
.........Parsing data...
Done!  Elapsed time: 18.9 seconds
Saving numpy and json to derived/2018-12-11/organoid-2-recording_3-wt_100hz-filter_181208_135711

original/2018-12-11/organoid 2/recording_3 wt_181208_135431.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
16 amplifier channels
3 auxiliary input channels
1 supply voltag

.........Parsing data...
Done!  Elapsed time: 26.5 seconds
Saving numpy and json to derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142327

original/2018-12-11/organoid 3/recording_4_wt_40mM KCl_181208_142427.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
16 amplifier channels
3 auxiliary input channels
1 supply voltage channels
0 board ADC channels
0 board digital input channels
0 board digital output channels
1 temperature sensors channels
File contains 34.776 seconds of data.  Amplifiers were sampled at 20.00 kS/s.

Allocating memory for data...
Reading data from file...
.........Parsing data...
Done!  Elapsed time: 15.3 seconds
Saving numpy and json to derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142427

original/2018-12-11/organoid 4/recording_4_wt 40mM KCl_181208_143057.rhd

Reading Intan Technologies RHD2000 Data File, Version 1.5

n signal groups 7
16 amplifier channels
3 auxiliary input channels
1 supply voltage c

In [38]:
!ls -alh "/public/groups/braingeneers/archive/derived/2018-12-11"

total 807M
drwxr-sr-x 2 30059 614 8.0K Jan  4 16:26 .
drwxr-sr-x 3 30059 614   32 Jan  4 15:49 ..
-rw-r--r-- 1 30059 614 7.6K Jan  4 16:16 organoid-1-recording-background-100Hz-filter_181208_130030.json
-rw-r--r-- 1 30059 614  26M Jan  4 16:16 organoid-1-recording-background-100Hz-filter_181208_130030.npy
-rw-r--r-- 1 30059 614 8.0K Jan  4 16:17 organoid-1-recording_1_181208_122247.json
-rw-r--r-- 1 30059 614  39M Jan  4 16:17 organoid-1-recording_1_181208_122247.npy
-rw-r--r-- 1 30059 614 8.0K Jan  4 16:17 organoid-1-recording_1_181208_122347.json
-rw-r--r-- 1 30059 614 2.6M Jan  4 16:17 organoid-1-recording_1_181208_122347.npy
-rw-r--r-- 1 30059 614 8.0K Jan  4 16:17 organoid-1-recording_2_100Hz-high-pass_181208_122907.json
-rw-r--r-- 1 30059 614  39M Jan  4 16:17 organoid-1-recording_2_100Hz-high-pass_181208_122907.npy
-rw-r--r-- 1 30059 614 8.0K Jan  4 16:17 organoid-1-recording_2_100Hz-high-pass_181208_123007.json
-rw-r--r-- 1 30059 614 3.1M Jan  4 16:17 organoid-1-recording_2_100

In [46]:
derived = np.load("derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.npy")
derived

array([[32715, 32698, 32695, ..., 32774, 32801, 32803],
       [32784, 32760, 32739, ..., 32781, 32781, 32769],
       [32730, 32687, 32696, ..., 32748, 32776, 32787],
       ...,
       [32773, 32793, 32796, ..., 32750, 32755, 32761],
       [32789, 32800, 32800, ..., 32746, 32742, 32764],
       [32841, 32826, 32862, ..., 32578, 32571, 32570]], dtype=uint16)

In [40]:
# Upload all of the .npy files to S3 replacing spaces with -
for path in glob.glob("derived/2018-12-11/*.*"):
    !aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
        s3 cp '{path}' s3://braingeneers/archive/{path} --acl public-read

upload: derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.npy to s3://braingeneers/archive/derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.npy
upload: derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.json to s3://braingeneers/archive/derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.json
upload: derived/2018-12-11/organoid-1-recording_1_181208_122247.npy to s3://braingeneers/archive/derived/2018-12-11/organoid-1-recording_1_181208_122247.npy
upload: derived/2018-12-11/organoid-1-recording_1_181208_122247.json to s3://braingeneers/archive/derived/2018-12-11/organoid-1-recording_1_181208_122247.json
upload: derived/2018-12-11/organoid-1-recording_1_181208_122347.npy to s3://braingeneers/archive/derived/2018-12-11/organoid-1-recording_1_181208_122347.npy
upload: derived/2018-12-11/organoid-1-recording_1_181208_122347.json to s3://braingeneers/archive/derived/2018-12-11/organo

upload: derived/2018-12-11/organoid-2-recording_3-wt_low-freq-noise_2_181208_140851.json to s3://braingeneers/archive/derived/2018-12-11/organoid-2-recording_3-wt_low-freq-noise_2_181208_140851.json
upload: derived/2018-12-11/organoid-2-recording_background-before-organoid-2_181208_130214.npy to s3://braingeneers/archive/derived/2018-12-11/organoid-2-recording_background-before-organoid-2_181208_130214.npy
upload: derived/2018-12-11/organoid-2-recording_background-before-organoid-2_181208_130214.json to s3://braingeneers/archive/derived/2018-12-11/organoid-2-recording_background-before-organoid-2_181208_130214.json
upload: derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142127.npy to s3://braingeneers/archive/derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142127.npy
upload: derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142127.json to s3://braingeneers/archive/derived/2018-12-11/organoid-3-recording_4_wt_40mM-KCl_181208_142127.json
upload: deriv

In [43]:
!aws --profile {os.getenv("AWS_PROFILE")} --endpoint {os.getenv("AWS_S3_ENDPOINT")} \
    s3 ls --recursive s3://braingeneers/archive/derived/2018-12-11

2019-01-04 16:40:33       7736 archive/derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.json
2019-01-04 16:40:32   26311808 archive/derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.npy
2019-01-04 16:40:37       8126 archive/derived/2018-12-11/organoid-1-recording_1_181208_122247.json
2019-01-04 16:40:36   40808288 archive/derived/2018-12-11/organoid-1-recording_1_181208_122247.npy
2019-01-04 16:40:40       8126 archive/derived/2018-12-11/organoid-1-recording_1_181208_122347.json
2019-01-04 16:40:39    2717408 archive/derived/2018-12-11/organoid-1-recording_1_181208_122347.npy
2019-01-04 16:40:44       8143 archive/derived/2018-12-11/organoid-1-recording_2_100Hz-high-pass_181208_122907.json
2019-01-04 16:40:42   40808288 archive/derived/2018-12-11/organoid-1-recording_2_100Hz-high-pass_181208_122907.npy
2019-01-04 16:40:46       8144 archive/derived/2018-12-11/organoid-1-recording_2_100Hz-high-pass_181208_123007.json
2019-01-04 1

In [47]:
# Read a single numpy file in directly from S3 to verify its the same as above
with np.DataSource(None).open("https://s3.nautilus.optiputer.net/braingeneers/archive/derived/2018-12-11/organoid-1-recording-background-100Hz-filter_181208_130030.npy", "rb") as f:
    signal = np.load(f)

signal == derived

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])